##Alunos: André Luiz Bacelar, Felipe Moreno, Gabriel Costa e Lucas Barros


#Instalação de dependências

In [ ]:
!pip install -U transformers datasets rouge-score nltk sentencepiece kaggle kagglehub evaluate torch==2.6.0 torchvision torchaudio  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import pandas as pd
import os
import torch
import numpy as np
import evaluate
import kagglehub
from datasets import Dataset, load_from_disk, load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          TrainingArguments, Trainer, DataCollatorForSeq2Seq)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando:", device)

Usando: cuda


#Configurar acesso ao Kaggle

In [ ]:
path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/newspaper-text-summarization-cnn-dailymail


#Carregar e preparar os dados

In [ ]:
dataset = load_dataset("csv", data_files={
    "train": "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv",
    "validation": "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv",
    "test": "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv"
})

train_ds = dataset["train"].select(range(5000))
val_ds   = dataset["validation"].select(range(500))
test_ds  = dataset["test"].select(range(500))

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# Pré-processamento e tokenização

In [ ]:
model_ckpt       = "t5-small"          # ou "facebook/bart-base"
tokenizer        = AutoTokenizer.from_pretrained(model_ckpt)
max_input_len    = 512
max_target_len   = 80

def preprocess(batch):
    inputs = tokenizer(
        ["summarize: " + text for text in batch["article"]],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    targets = tokenizer(
        batch["highlights"],
        max_length=80,
        truncation=True,
        padding="max_length"
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
tokenized_train = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
tokenized_val   = val_ds.map(preprocess, batched=True, remove_columns=val_ds.column_names)

tokenized_train.save_to_disk("tokenized_train")
tokenized_val.save_to_disk("tokenized_val")

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

#Treinamento (Fine-Tuning)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,
    save_strategy="no",
    max_steps=200,
    report_to="none",
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train.select(range(2000)),
    eval_dataset=tokenized_val.select(range(200)),
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

<ipython-input-24-f9edcee19d43>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,4.424800
100,3.211000
150,2.545000
200,2.288600


TrainOutput(global_step=200, training_loss=3.117370491027832, metrics={'train_runtime': 29.1136, 'train_samples_per_second': 27.479, 'train_steps_per_second': 6.87, 'total_flos': 108273441177600.0, 'train_loss': 3.117370491027832, 'epoch': 0.4})

#Avaliação

In [ ]:
import random

rouge = evaluate.load("rouge")

def evaluate_random_prediction():
    i = random.randint(0, len(test_ds) - 1)
    article = test_ds[i]["article"]
    reference = test_ds[i]["highlights"]

    inputs = tokenizer("summarize: " + article,
                       return_tensors="pt",
                       truncation=True,
                       max_length=512).input_ids.to(model.device)

    output = model.generate(inputs, max_length=60)
    generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)

    # Calcular ROUGE
    scores = rouge.compute(predictions=[generated_summary], references=[reference])

    print(f"\nNotícia aleatória (#{i}):\n", article[:300], "...\n")
    print("Resumo gerado:\n", generated_summary)
    print("Resumo referência:\n", reference)
    print("\n ROUGE scores:")
    for key, value in scores.items():
        print(f"{key}: {value:.4f}")

evaluate_random_prediction()


Notícia aleatória (#327):
 A company has released matches that continue to burn even when submerged in water or buried underground in dirt. The UCO Stormproof Matches are made using an incredibly tough coating that smoulders no matter what happens to them and will start burning again once they come into contact with oxygen. T ...

Resumo gerado:
 the UCO Stormproof Matches are made by a company that has been making outdoor accessories for the past 44 years. they include 25 matches in a completely waterproof case with and three strike pads. they are available from the UK distributor Dalesman International.
Resumo referência:
 Washington company unveils matches that burn in testing conditions .
The £5.47 UCO Stormproof Matches use a coating that always smoulders .
They will start burning again once they comes into contact with oxygen .
Rigorous tests have drenched the matches in buckets, buried them in mud, and blown them with a compressed air hose .

 ROUGE scores:
rouge1: 0.3125
rouge